# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import prince
import seaborn as sns
import statsmodels.api as sm
from mlxtend.plotting import plot_pca_correlation_graph
from scipy.stats import bartlett, shapiro
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.formula.api import ols

In [ ]:
SEED = 0

In [ ]:
np.random.seed(SEED)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
df_transforme = pd.read_csv(
    "data/data-transformed.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

# Clusters etc

## ACM

In [ ]:
mca = prince.MCA(n_components=df[var_categoriques].shape[1], random_state=SEED)

In [ ]:
mca.fit(df[var_categoriques])

In [ ]:
X_clust = mca.row_coordinates(df[var_categoriques])

In [ ]:
X_clust.columns = df[var_categoriques].columns

In [ ]:
X_clust.head()

## Fusion ACM et var quanti

In [ ]:
X_clust = pd.concat((df[var_numeriques], X_clust), axis=1)

In [ ]:
X_clust.head()

In [ ]:
scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(X_clust),
    columns=X_clust.columns,
)

In [ ]:
df_apres_scale.head()

## Différents algorithmes de clusters

In [ ]:
model_clusters = {
    "KMeans2": KMeans(n_clusters=2, random_state=SEED),
    "KMeans3": KMeans(n_clusters=3, random_state=SEED),
    "KMeans4": KMeans(n_clusters=4, random_state=SEED),
    "GMM2": GaussianMixture(n_components=2, covariance_type="full", random_state=SEED),
    "GMM3": GaussianMixture(n_components=3, covariance_type="full", random_state=SEED),
    "GMM4": GaussianMixture(n_components=4, covariance_type="full", random_state=SEED),
    "DBSCAN": DBSCAN(),
    "OPTICS": OPTICS(),
    "CAH": DBSCAN(),  # todo: CAH
}

In [ ]:
for (model_name, model) in model_clusters.items():
    if model.__class__
    model.fit()

In [ ]:
k2 = KMeans(n_clusters=2)
k3 = KMeans(n_clusters=3)
k4 = KMeans(n_clusters=4)

In [ ]:
k2.fit(df_apres_scale)
k3.fit(df_apres_scale)
k4.fit(df_apres_scale)

In [ ]:
df_apres_scale["cluster_k2"] = k2.labels_
df_apres_scale["cluster_k3"] = k3.labels_
df_apres_scale["cluster_k4"] = k4.labels_

## Visualisation

In [ ]:
def affiche_taille_clusters(nom_cluster):
    plt.title("Taille des clusters")
    sns.histplot(df_apres_scale[nom_cluster], shrink=0.5)

    plt.show()

In [ ]:
def affiche_clusters_var_quanti(nom_cluster):
    """Affiche les variables quantitatives en fonction des clusters."""
    for var in var_numeriques:
        _, ax = plt.subplots(1, 2, figsize=(10, 3))

        sns.boxplot(
            x=df[var],
            y=df_apres_scale[nom_cluster],
            width=0.25,
            ax=ax[0],
        )

        sns.histplot(
            x=df[var],
            kde=True,
            ax=ax[1],
            hue=df_apres_scale[nom_cluster],
            stat="probability",
            common_norm=False,
        )

        plt.show()

In [ ]:
def affiche_clusters_var_quali(nom_cluster):
    """Affiche les variables qualitatives en fonction des clusters et vice-versa."""
    for var in var_categoriques:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))

        sns.histplot(
            x=df[var].astype(str),
            ax=ax[0],
            hue=df_apres_scale[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        sns.histplot(
            hue=df[var].astype(str),
            ax=ax[1],
            x=df_apres_scale[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        plt.show()

In [ ]:
def affiche_clusters(nom_cluster):
    """Affiche les variables en fonction des clusters."""
    # todo: modifier les clusters avant la fonction
    df_apres_scale[nom_cluster] = pd.Categorical(
        df_apres_scale[nom_cluster].astype(str)
    )

    affiche_taille_clusters(nom_cluster)
    affiche_clusters_var_quanti(nom_cluster)
    affiche_clusters_var_quali(nom_cluster)

In [ ]:
affiche_clusters("cluster_k2")

In [ ]:
affiche_clusters("cluster_k3")

In [ ]:
affiche_clusters("cluster_k4")

## Métriques d'évaluation des clusters

https://scikit-learn.org/stable/modules/classes.html#clustering-metrics
https://heartbeat.comet.ml/how-to-evaluate-clustering-based-models-in-python-503343816db2

In [ ]:
df_sans_cluster = df_apres_scale.drop(
    columns=["cluster_k2", "cluster_k3", "cluster_k4"]
)  # todo: mettre de meilleurs noms

In [ ]:
cluster_metrics = []
clusters = [k2, k3, k4]

for c in clusters:
    cluster_metrics.append(
        [
            silhouette_score(df_sans_cluster, c.labels_),  # proche de 1 = mieux
            calinski_harabasz_score(
                df_sans_cluster, c.labels_
            ),  # plus élevé, mieux c'est
            davies_bouldin_score(df_sans_cluster, c.labels_),  # proche de 0 = mieux
        ]
    )

In [ ]:
pd.DataFrame(
    cluster_metrics,
    columns=["Silhouette", "Calinski-Harabasz", "Davies-Bouldin"],
    index=["Kmeans n=2", "Kmeans n=3", "Kmeans n=4"],
)

In [ ]:
pd.DataFrame(
    np.array(["n=3", "n=2", "n=3"]).reshape((1, 3)),
    columns=["Silhouette", "Calinski-Harabasz", "Davies-Bouldin"],
    index=["Nombre optimal de clusters"],
)

# Autres clusters

## DBScan

In [ ]:
db = DBSCAN()
optics = OPTICS()

In [ ]:
db.fit(df_sans_cluster)
optics.fit(df_sans_cluster)

In [ ]:
plt.hist(db.labels_)

In [ ]:
plt.hist(optics.labels_)

## GMM

In [ ]:
gmm2.__class__ == GaussianMixture

In [ ]:
gmm2 = GaussianMixture(n_components=2, covariance_type="full")
gmm3 = GaussianMixture(n_components=3, covariance_type="full")
gmm4 = GaussianMixture(n_components=4, covariance_type="full")

In [ ]:
labels_gmm2 = gmm2.fit_predict(df_sans_cluster)
labels_gmm3 = gmm3.fit_predict(df_sans_cluster)
labels_gmm4 = gmm4.fit_predict(df_sans_cluster)

In [ ]:
cluster_metrics = []
clusters = [labels_gmm2, labels_gmm3, labels_gmm4]

for c in clusters:
    cluster_metrics.append(
        [
            silhouette_score(df_sans_cluster, c),  # proche de 1 = mieux
            calinski_harabasz_score(df_sans_cluster, c),  # plus élevé, mieux c'est
            davies_bouldin_score(df_sans_cluster, c),  # proche de 0 = mieux
        ]
    )

In [ ]:
pd.DataFrame(
    cluster_metrics,
    columns=["Silhouette", "Calinski-Harabasz", "Davies-Bouldin"],
    index=["GMM n=2", "GMM n=3", "GMM n=4"],
)

In [ ]:
pd.DataFrame(
    np.array(["n=2", "n=2", "n=2"]).reshape((1, 3)),
    columns=["Silhouette", "Calinski-Harabasz", "Davies-Bouldin"],
    index=["Nombre optimal de clusters"],
)

# Analyse multi-variée

## Analyse en Composantes Principales (ACP)

In [ ]:
# todo: centrer / réduire

In [ ]:
scaler = StandardScaler()

In [ ]:
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df[var_numeriques]), columns=df[var_numeriques].columns
)

In [ ]:
df_apres_scale

In [ ]:
acp = PCA(random_state=SEED)

In [ ]:
acp.fit(df_apres_scale)

In [ ]:
variance_expliquee = pd.Series(
    acp.explained_variance_ratio_, index=df[var_numeriques].columns
)

In [ ]:
variance_expliquee

In [ ]:
variance_expliquee.plot.barh()

In [ ]:
df_acp = pd.DataFrame(acp.fit_transform(df_apres_scale), index=df.index)

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1)

### Cercle de corrélations

In [ ]:
_, correlation_matrix = plot_pca_correlation_graph(
    df_apres_scale,
    df_apres_scale.columns,
    X_pca=df_acp.iloc[:, :2],
    explained_variance=acp.explained_variance_[:2],
    dimensions=(1, 2),
)

In [ ]:
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

### Test clusters

In [ ]:
df_acp["clusterk3"] = k3.labels_
df_acp["clusterk4"] = k4.labels_

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk3")

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=2, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk4")

## Analyse Factorielle des Correspondances (AFC)

In [ ]:
table_contingence = pd.crosstab(df["Kidhome"], df["Teenhome"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(
    # n_components=3,
    # n_iter=3,
    # copy=True,
    # check_input=True,
    # engine='sklearn',
    random_state=SEED
)

ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
table_contingence = pd.crosstab(df["Marital_Status"], df["Education"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(random_state=SEED)
ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
# todo: à interpréter

## Analyse des Correspondances Multiples (ACM)

In [ ]:
mca = prince.MCA(random_state=SEED)
mca = mca.fit(df[var_categoriques])

In [ ]:
mca.plot(df[var_categoriques])

In [ ]:
# todo: à interpréter

# ANOVA

## Problématique

Nous allons tester l'indépendance entre la variable `Response` (catégorique, binaire) et `Income` (quantitative continue), pour répondre à la question : "le revenu influence-t-il la réponse aux campagnes marketing ?".

In [ ]:
sns.boxplot(df, x="Response", y="Income")

On cherche à déterminer si les moyennes des groupes sont significativement différentes. On pose donc :

$H_0$ : Les moyennes de chaque groupe sont égales si la p-value $> 5\%$
$H_1$ : Les moyennes de chaque groupe ne sont pas toutes égales si la p-value $< 5\%$


## Hypothèses

1) l’indépendance entre les échantillons de chaque groupe
2) l’égalité des variances que l’on peut verifier avec un test de Bartlett.
3) la normalité des résidus avec un test de Shapiro.

### Indépendance

### Egalité des variances

Voici la variance de chaque groupe :

In [ ]:
df.groupby("Response")["Income"].agg("var")

Nous allons effectuer un test de Bartlett pour vérifier l'égalité des variances.

$H_0$ : Les variances de chaque groupe sont égales si p-value $> 5\%$
$H_1$ : Les variances de chaque groupe ne sont pas toutes égales $< 5\%$


In [ ]:
bartlett(
    df["Income"][df["Response"] == "0"],
    df["Income"][df["Response"] == "1"],
)

Notre p-value est inférieure à 5%, donc les variances ne sont pas toutes égales au (risque de 5%).

### Normalité des résidus

Nous utilisons le test de Shapiro-Wilk pour vérifier la normalité des résidus.

H0 : Les résidus suivent une loi normale si p-value > 5%
H1 : Les résidus ne suivent pas une loi normale si p-value < 5%

In [ ]:
model = ols("Income ~ Response", data=df).fit()
shapiro(model.resid)

Comme la p-value est inférieure à 5%, les résidus ne suivent pas une loi normale (au risque de 5%).

## Test d'ANOVA

H0 : Les moyennes de chaque groupe sont égales si p-value > 5%
H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

In [ ]:
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

Comme aucune hypothèse n'est vérifiée, le test n'est pas fiable.

In [ ]:
# todo: tenter le GLS

# Sauvegarde du Dataframe

In [ ]:
cluster3 = pd.DataFrame(k3.labels_, columns=["cluster3"])
cluster4 = pd.DataFrame(k4.labels_, columns=["cluster4"])

In [ ]:
cluster3.to_csv("data/clusters-kmeans3.csv")
cluster4.to_csv("data/clusters-kmeans4.csv")